In [41]:
import urllib.request
import requests
from bs4 import BeautifulSoup
import pandas as pd
import urltools
import time 
from pandas import ExcelWriter
from pandas import ExcelFile



def get_base_url(url):
    from urllib.parse import urlsplit
    base_url = "{0.scheme}://{0.netloc}/".format(urlsplit(url))
    

    if base_url.startswith("https"):
        base_url = base_url.replace('https://','')
    elif base_url.startswith("http"):
        base_url = base_url.replace('http://','')
        
    if base_url.endswith('/'):
        base_url = base_url.replace('/','')
    return base_url



def does_page_exist(url):
    exists = 0 
    try:
        request = requests.get(url)
        if request.status_code == 200:
            exists = 1 
            
    except: 
        pass
    
    return exists


# Function to extract links from a URL------------------------------------------
def href_scrapper(url):

    try:
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        a_tag = soup.find_all('a')
        img_tag = soup.find_all('img')
        link_tag = soup.find_all('link')
        link_list = list()

        for links in a_tag:
            link_list.append(links.get('href'))

        for links in link_tag:
            link_list.append(links.get('href'))

        for links in img_tag:
            link_list.append(links.get('src'))

        return link_list

    except:
        return 0


# Function to create complete links---------------------------------------------
def link_former(url):
    x = url

    if x.startswith("http"):
        return x

    else:
        full_url = original_url + x
        return full_url




# Function to save links to directory of python file----------------------------
def file_save(url, i):
    save_error = 0
    full_url = url
    try:
        if str(full_url).endswith(".htm"):
            name = "WebPage" + str(i) + ".htm"
            urllib.request.urlretrieve(full_url, name)
            m = i + 1

        elif str(full_url).endswith(".pdf"):
            name = "Pdf" + str(i) + ".pdf"
            urllib.request.urlretrieve(full_url, name)
            m = i + 1

        elif str(full_url).endswith(".jpg"):
            name = "Image" + str(i) + ".jpg"
            urllib.request.urlretrieve(full_url, name)
            m = i + 1

        elif str(full_url).endswith(".jpeg"):
            name = "Image" + str(i) + ".jpeg"
            urllib.request.urlretrieve(full_url, name)
            m = i + 1

        elif str(full_url).endswith(".png"):
            name = "Image" + str(i) + ".png"
            urllib.request.urlretrieve(full_url, name)
            m = i + 1

        elif str(full_url).endswith(".css"):
            name = "CSS File" + str(i) + ".css"
            urllib.request.urlretrieve(full_url, name)
            m = i + 1

        else:
            name = "WebPage" + str(i) + ".html"
            urllib.request.urlretrieve(full_url, name)
            m = i + 1

        return m
    except:
        save_error = save_error + 1




# In[7]:

def crawl(original_url, num_Id, output_file):
    
    # Accepting input of URL and depth----------------------------------------------
    tic = time.time()
    
    parent_list = [original_url]
    url_to_check = get_base_url(original_url)
    print(url_to_check)
    url_to_check = str(url_to_check)

    layer_stop = 1
    layer = 0


    #initializing all the requiered lists---------------------------------------------
    visited_all  = [original_url]
    visited_current_layer = []
    child_list =[]
    child_list_filtered = []
    #columns = ['Link','Parent Link', 'Layer']
    df = pd.DataFrame()


    # Main execution of scrapper----------------------------------------------------

    #looping through layers 
    while layer < layer_stop:
    
        #looping through URLs in parent-list
        for url in parent_list:
        
            #scraping the children from the parent url----------------------------
            if href_scrapper(url) != 0:
                child_list = href_scrapper(url)
    
    
            for child in child_list:
                if child != None:
                    #if child link is of the form "index.php/blahblah" and parent ends with '/'
                    #---> "parentlink/index.php/blahblah"
                    if child.startswith('/'):
                        child= str(url) + str(child)
                    
                    if url.endswith('/') and url_to_check not in child:
                        child = str(url) + str(child)
                    
                #normalize the child links-------------------------------------
                    child=urltools.normalize(child)  
                    

                    #filtering out based on 1) External 2) Repeating 3) Invalid links---------------------------
                    if url_to_check in child and child not in visited_all and does_page_exist(child)==1:
                        child_list_filtered.append(child)
                
                    #adding everthing to visited all--------------------
                    if child not in visited_all:
                        child_slash = child + '/'
                        visited_all.append(child)  
                        visited_all.append(child_slash)
                    
                    
            #adding  the visited and filtered children into the "current visited layer" ------ 
            for child_filtered in child_list_filtered:
                visited_current_layer.append(child_filtered)

            #creating a Pandas dataframe to store everything for download----------
            layer_number = [layer+1]*len(child_list_filtered)
            parent_of_child = [url]*len(child_list_filtered)

            df_child = pd.DataFrame(child_list_filtered)
            df_parent = pd.DataFrame(parent_of_child)
            df_layer = pd.DataFrame(layer_number)


            df_to_be_added = pd.concat([df_child,df_parent,df_layer], axis=1)
            df = pd.concat([df,df_to_be_added],ignore_index=True, axis = 0)
            #----------------------------------------------------------------------
        
            #emptying the child lists
            child_list = []
            child_list_filtered = []
        
        #condition to stop filtering-----------------------------------------------
        if not visited_current_layer :
            layer_stop = layer_stop 
        else:
            layer_stop += 1

    
        #child layer is now parent layer--------------------------------------------
        parent_list = []
    
        #we only dont add .png, .jpg , .pdf to the new parent layer 
        for visited_current in visited_current_layer: 
            print(visited_current)
            if(not visited_current.endswith('.png') and not visited_current.endswith('.jpg') and not  visited_current.endswith('.pdf')):
                parent_list.append(visited_current)
            
            
        #displaying the links in different layers----------------------------------
        #print("Links in LAYER:" + str(layer+1))
        print("No of links = " + str(len(visited_current_layer)))
        #print(visited_current_layer)
        print("\n")
        visited_current_layer = [] 
        #updating the layer number
        layer +=1
    df.to_csv(output_file + '/' + str(num_Id) + '_' + str(url_to_check) +  '.csv', sep=',', encoding='utf-8')
    return df, num_Id


In [42]:
import requests

page = requests.get("https://docs.python.org/3/howto/urllib2.html")
from bs4 import BeautifulSoup
from bs4.element import Comment
soup = BeautifulSoup(page.content, 'html.parser')


#information based on:
def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True
p_tag=soup.find_all('p')

#extract text from html: 
def text_from_html(body):
    soup = BeautifulSoup(body, 'html.parser')
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)  
    return u" ".join(t.strip() for t in visible_texts)
web_text = []


In [43]:
import json
import urllib
import os 

def scrape(df, scraped_website_path, website_index, website_name):
    TempD={}
    for index,item in df.iterrows():
        try:      
            print(item[1])
            html = urllib.request.urlopen(str(item[1])).read()
            #web_text.append(text_from_html(html))
            print(text_from_html(html))
            #save_url(str(item[1]), index)
            TempD[str(item[1])]=text_from_html(html)
        except: 
            pass
    with open(scraped_website_path + '/' + 'json_' + str(website_index) + str(get_base_url(website_name)) +  '.json', 'w') as outfile:  
        json.dump(TempD, outfile)
        
    return 1      

### Naming conventions 

#### Naming of csv files with child links,  we name the csv files with the child links :
```<index>_<parent website>.csv```

#### Naming of JSON files :
```<index>_<parent website>.json ```


In [44]:
# web_crawler_all() = Scraper for multiple websites
'''
Inputs
-----------
excel_path = Excel file path which contains list of websites to be crawled  
sheet = sheet name of this excel file 
web_col = the column no of in which the website hompages are located 
index_col = the column with indices of websites
parent_col = the column with the parent names
output_folder = path of the output folder where we wish to save the websites 
from_website = the index no. from website (typically starting from 1)
to_website = the index no of the website till we want to continue scraping (scraped websites will include the 
                websites with this index)

Output 
-----------
CSV files containing a list of child links for each website are saved into the output_folder path 

'''
def web_crawler_all(excel_path, sheet, web_col, index_col, parent_col,  output_folder, from_website, to_website):
    df = pd.read_excel(excel_path , sheet_name=sheet)
    df = df.iloc[from_website-1 : to_website, :]
    print(df)
    for index,row in df.iterrows():
        num_Id=row[index_col]
        original_url=row[web_col]
        print("Website no : " + str(num_Id))
        print("Website name : " + str(original_url))
        
        try:
            websites= pd.read_csv(output_folder + str(num_Id) + '_' + str(original_url) + '.csv')
            print("website is already crawled")
        except:
            websites, num_Id = crawl(original_url ,num_Id ,output_folder)
            
        
    print("All links from the websites indicated have been crawled")
    print("Output File :  " + str(output_folder))
    print("No. of websites scraped : " + str(to_website - from_website + 1))
    return 1

In [45]:
#executing the crawler function 

excel_path = 'C:/Users/mihirbhatia999/Desktop/DCMME/scraped_data/trial1.xlsx'
sheet = 'From List of Companies'
web_col = 2
index_col = 0
parent_col = 1
output_folder = 'C:/Users/mihirbhatia999/Desktop/DCMME/scraped_data/crawled_websites'
from_website = 1 
to_website = 2 

web_crawler_all(excel_path,sheet, web_col, index_col, parent_col,output_folder, from_website, to_website)
    

   Serial No.   Company Name                      Website  \
0           1  Mihir Bhatia   https://www.mihirbhatia.com   
1           2      Ajin Tom       https://www.ajintom.com   

   Has a Website?\n(1=Yes, 0=No) Static/Dynamic Website?  \
0                              1                       D   
1                              1                       D   

   If no website- Is present on Facebook?\n(1=Yes, 0=No)  
0                                                NaN      
1                                                NaN      
Website no : 1
Website name : https://www.mihirbhatia.com
www.mihirbhatia.com
https://www.mihirbhatia.com/s/curriculum-vitae-5.pdf
https://www.mihirbhatia.com/bio/
https://www.mihirbhatia.com/experience/
https://www.mihirbhatia.com/projects/
No of links = 4


No of links = 0


Website no : 2
Website name : https://www.ajintom.com
www.ajintom.com
No of links = 0


All links from the websites indicated have been crawled
Output File :  C:/Users/mihirbhatia9

1

In [46]:
'''
Inputs 
----------
excel_path = Excel file path which contains list of websites to be crawled  
sheet = sheet name of this excel file 
crawled_website_path = path of file where all the crawled websites are stored  
scraped_website_path = path of file where all the scraped websites are stored
from_website - to_website = website index number from-to which we want to scrape and store in JSON 
web_col = column number of websites (0,1,2....)

Output
----------
JSON files for each parent website in scraped_website_path folder. The path of the scraped websites 
is the same as that crawled websites 
'''

def web_scraper_all(excel_path, sheet, crawled_website_path, scraped_website_path, from_website, to_website,web_col):
    #get excel sheet containing websites 
    df = pd.read_excel(excel_path , sheet_name=sheet)
    df = df.iloc[from_website-1 : to_website, :]   
    
    #loop over all the websites and extract child links. Store them in JSON files 
    for i in range(from_website, to_website+1):
        #setting the website index and name 
        website_index = i 
        website_name = df.iloc[i-1,web_col]
        
        #generating filename and getting the csv file of child links 
        filename = crawled_website_path + '/'+ str(website_index)+ '_' + str(get_base_url(website_name)) + ".csv"
        df_of_child_links = pd.read_csv(filename)
        
        #executing the scrape function 
        flag = scrape(df_of_child_links , scraped_website_path, website_index, website_name) 
        
        #checkpoint to see if scraping is complete 
        if flag == 1 :
            print(website_name + ' has been scraped successfully')
            print('scraping percentage complete = ' + str((i/to_website)*100) + '%')
            flag = 0 
        


In [47]:
#executing the scraper function 

excel_path = 'C:/Users/mihirbhatia999/Desktop/DCMME/scraped_data/trial1.xlsx'
sheet = 'From List of Companies'
crawled_website_path = 'C:/Users/mihirbhatia999/Desktop/DCMME/scraped_data/crawled_websites'
scraped_website_path = 'C:/Users/mihirbhatia999/Desktop/DCMME/scraped_data/scraped_websites'
from_website = 1
to_website = 1 
to_website = 2
web_col = 2 

web_scraper_all(excel_path, sheet, crawled_website_path, scraped_website_path, from_website, to_website,web_col)

https://www.mihirbhatia.com/s/curriculum-vitae-5.pdf
https://www.mihirbhatia.com/bio/
https://www.mihirbhatia.com/experience/
https://www.mihirbhatia.com/projects/
            RESUME  Bio  HOME  EXPERIENCE  PROJECTS      RESUME  Bio  HOME  EXPERIENCE  PROJECTS                                                         ASPIRING DATA SCIENTIST | RESEARCHER | MECHANICAL ENGINEER          Select project category   DATA SCIENCE    INFORMATION ENGINEERING    MACHINE VISION    ROBOTICS  DATA SCIENCE Some of the instances that I've played around with data: High Energy Physics particle tracking in CERN detectors (Kaggle) - Ongoing Stock Portfolio Performance by Weighted Stock Selection Prediction of Electricity used for space conditioning for Buildings in the Pacific Region Prediction of Electricity costs (using Regression, Random Forests and MARS)  High Energy Physics particle tracking in CERN detectors (Kaggle)         A Kaggle competition seemed like the appropriate challenge for the Summer. We

In [4]:
import json
from pprint import pprint

with open('json1.json') as f1:
    data1 = json.load(f1)
    
with open('json2.json') as f2:
    data2 = json.load(f2)
    



In [13]:
original_url = 'https://www.ajintom.com'
num_Id  = 1 
output_folder = 'C:/Users/mihirbhatia999/Desktop/DCMME/scraped_data/crawled_websites'
df, num_Id = crawl(original_url, num_Id, output_folder)

www.ajintom.com
https://www.ajintom.com/
https://www.ajintom.com/bio/
https://www.ajintom.com/music/
https://www.ajintom.com/tech/
https://www.ajintom.com/s/Ajin-Tom-Resume18.pdf
https://www.ajintom.com/contact/
No of links = 6


No of links = 0


